# Face detection

Cette fonctionnalité permet de détecter des visages humains sur une image et d'en extraire certaines caractéristiques comme la localisation sur l'image, l'âge prédit, l'émotion...

### Import des bibliothèques

In [1]:
import os
import requests
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

from json import dump

from dotenv import load_dotenv

import cv2 


### Chargement de la clé d'API et du endpoint

In [2]:
load_dotenv()

cog_endpoint = os.getenv("COG_SERVICE_ENDPOINT")
cog_key = os.getenv("COG_SERVICE_KEY")

### Prise de photo avec la webcam

In [ ]:
key = cv2. waitKey(1)
webcam = cv2.VideoCapture(0)
while True:
    try:
        check, frame = webcam.read()
        print(check) #prints true as long as the webcam is running
        print(frame) #prints matrix values of each framecd 
        cv2.imshow("Capturing", frame)
        key = cv2.waitKey(1)
        if key == ord('s'): 
            cv2.imwrite(filename='saved_img.jpg', img=frame)
            webcam.release()
            img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
            img_new = cv2.imshow("Captured Image", img_new)
            cv2.waitKey(1650)
            cv2.destroyAllWindows()
            print("Processing image...")
            img_ = cv2.imread('saved_img.jpg', cv2.IMREAD_ANYCOLOR)
            
            print("Image saved!")
        
            break
        elif key == ord('q'):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break
        
    except(KeyboardInterrupt):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

## Utilisation du module Detect de Face API

Objectif : détecter le ou les visage(s) présent(s) dans l'image et extraire pour chaque visage l'âge, le genre et l'émotion correspondants.

In [5]:
def detect_face(key, endpoint, img):
    # construct the API URL endpoint for the Language Detection service (use `cog_endpoint`)
    url = f"{endpoint}face/v1.0/detect"
    
    # fill the required headers to authenticate with the API (use `cog_key`)
    headers = {
        "Content-Type": "application/octet-stream",
        "Ocp-Apim-Subscription-Key": key,
    }

    with open(img,'rb') as image:
        
        # fill the request body parameters with the input text
        '''
        body = {
            "url": img
        }
        
        '''
        
        params = {
            "returnFaceAttributes": "age,gender,emotion",
            "recognitionModel": "recognition_04",
        }

        # make a POST request to the API using the url, the headers and the JSON body parameters
        response = requests.post(url, data=image, params=params, headers=headers)

        response.raise_for_status()
        data = response.json()

        
        # On récupère les attributs âge, genre, émotion ainsi que les coordonnées de la position pour chaque visage détecté
        face_att = []
        
        for face in data:
            rect = {}
            age = face['faceAttributes']['age']
            gender = face['faceAttributes']['gender']
            emotion = max(face['faceAttributes']['emotion'], key=face['faceAttributes']['emotion'].get)
            score_emotion = max(face['faceAttributes']['emotion'].values())
            rect['top'] = face['faceRectangle']["top"]
            rect['left'] = face['faceRectangle']["left"]
            rect['width'] = face['faceRectangle']["width"]
            rect['height'] = face['faceRectangle']["height"]
            face_att.append((age,gender,emotion,rect))
        
        # Afficher l'image avec le rectangle détectant les visages avec les annotations
        
        fig = plt.figure(figsize=(8, 6))
        plt.axis('off')
        image = Image.open(img)
        draw = ImageDraw.Draw(image)
        color = 'red'

        for face in face_att:

                # Get face properties
                    
                print(' - Age: {}'.format(face[0]))
                            
                print(' - Emotion: {}'.format(face[2]))
                        
                print(' - Gender:{}'.format(face[1]))
                        
                print()

                # Draw and annotate face
                age = face[0]
                gender = face[1]
                emotion = face[2]
                left = face[3]['left']
                top = face[3]['top']
                width = face[3]['width']
                height = face[3]['height']
                bounding_box = ((left,top), (left + width, top + height))
                draw = ImageDraw.Draw(image)
                        
                if gender == 'female':
                    draw.rectangle(bounding_box, outline='green', width=4)
                else:
                    draw.rectangle(bounding_box, outline='red', width=4)
                            
                annotation = age
                        
                if gender == 'female':
                    plt.annotate(annotation,(left, top), backgroundcolor='green')
                    plt.annotate(f"{emotion} : {score_emotion}",(left+width/4, top+height*1.1), backgroundcolor='green')
                else:
                    plt.annotate(annotation,(left, top), backgroundcolor='red')
                    plt.annotate(f"{emotion} : {score_emotion}",(left+width/4, top+height*1.2), backgroundcolor='red')

        # Save annotated image
        plt.imshow(image)
        outputfile = 'detected_faces.jpg'
        fig.savefig(outputfile)

        print('\nResults saved in', outputfile)

        return face_att

In [ ]:
faces = detect_face(cog_key, cog_endpoint, 'surprise.jpg')

In [ ]:
faces